<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [2]:
import numpy as np

In [3]:
x = np.array([[0,0,1],
             [0,1,1],
             [1,0,1],
             [0,1,0],
             [1,0,0],
             [1,1,1],
             [0,0,0]])

y = np.array([[0],
             [1],
             [1],
             [1],
             [1],
             [0],
             [0]])

In [18]:
class MultiPerceptron:
    def __init__(self):
        # Set up Architecture of Multilayer Perceptron
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)
        
    def predict(self, X):
        o = self.feed_forward(X)
        return np.where(o <= 0.5, 1, 0)

mp = MultiPerceptron()

In [19]:
mp.train(x, y)

In [22]:
x[1]

array([0, 1, 1])

In [23]:
mp.predict(x[1])

array([0])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28


In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [6]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [7]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [8]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

In [240]:
class Neuron(object):    
    def __init__(self, weight, act_func=None, act_func_prime=None):
        self.weight = weight
        
        if act_func==None and act_func_prime==None:
            self.act_func = self.sigmoid
            self.act_prime = self.sigmoidPrime
        elif act_func==None or act_func_prime==None:
            raise TypeError("Must define both act_func and act_func_prime for custom functions")
        else:
            self.act_func = act_func
            self.act_prime = act_func_prime
            
    # Default Activation Function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # Default Activation Function Derivative
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    # Class Representation
    def __repr__(self):
        o = """
        weight = {},
        act_func = Memory Location {},
        act_func_prime = Memory Location {}
        """.format(self.weight, hex(id(self.act_func)), hex(id(self.act_prime)))
        return o
        

In [240]:
class NodeLayer(object):
    def __init__(self, NumNodes = 1, **kwargs):
        self.nodes = self.__gen_nodes(count = NumNodes, **kwargs)
        self.__weights = [x.weight for x in self.nodes]
    
    #Method to generate NumNodes of neurons
    def __gen_nodes(self, count, weight=None, act_func=None, act_func_prime=None):
        list_of_nodes = []
        for x in range(0,count):
            if weight == None:
                w = np.random.rand(1)[0]
                if act_func==None and act_func_prime==None:
                    t = Neuron(weight = w)
                else:
                    t = Neuron(weight = w, act_func = act_func, act_func_prime= act_func_prime)
            else:
                if act_func==None and act_func_prime==None:
                    t = Neuron(weight = weight[x])
                else:
                    t = Neuron(weight = weight[x], act_func = act_func, act_func_prime= act_func_prime)
                    
            list_of_nodes.append(t)
        return list_of_nodes
    
    #Accessor Methods
    def set_weights(self, new_weights):
        self.__weights = new_weights
        self.__update_weights()
        
    def add_weights(self, weights):
        self.__weights += weights
        self.__update_weights()
    
    # Utility Methods
    def __update_weights(self):
        for old,new in zip(self.nodes, self.__weights):
            old.weight = new
    
    #Representation Methods 
    def __repr__(self):
        o = ""
        for x,count in zip(self.nodes, range(0,len(self.nodes))):
            o = o + "Node #{}: {}\n".format(count, x)
        return o
    
    def __str__(self):
        return self.__repr__()
        

In [241]:
nl = NodeLayer(NumNodes=2)

In [242]:
nl

Node #0: 
        weight = 0.9440553895457546,
        act_func = Memory Location 0x20a0bd94308,
        act_func_prime = Memory Location 0x20a0be5f6c8
        
Node #1: 
        weight = 0.7641132654476426,
        act_func = Memory Location 0x20a0bd91c48,
        act_func_prime = Memory Location 0x20a098aad08
        

### Estimating Your `net

In [144]:
##### Your Code Here #####
import numpy as np

class Neural_Network(object):
    def __init__(self):        
        #Define Hyperparameters
        self.inputLayerSize = 784
        self.outputLayerSize = 1
        self.hiddenLayerSize = 1000
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-144-037d3a37fc32>, line 16)

In [48]:
from scipy import optimize
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()
        
        options = {'maxiter': 20, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [49]:
NN = Neural_Network()
T = trainer(NN)

In [50]:
x = x_train[:1000]
y = y_train[:1000]

In [51]:
y[1:10]

array([1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [52]:
T.train(x, y)

ValueError: shapes (1000,1000) and (1,1000) not aligned: 1000 (dim 1) != 1 (dim 0)

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?